In [4]:
import torch.nn as nn
import torch
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import random
from tqdm import tqdm
import numpy as np
import pickle

In [5]:
os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/")
filename = "EuroparlNutidsr_trainset_verbs.csv"
print("Loading df...")
df = pd.read_csv(filename, encoding="UTF-8", sep=";")
pos = list(df["comment_text"].values)
labels = list(df["label"].values)

X_train, X_test, y_train, y_test = train_test_split(pos, labels, test_size=0.1, random_state=42)

unique_pos = ['NOUN','PUNCT','VERB','PRON','NUM','ADP','X','<PAD>','CCONJ','PROPN','AUX','SCONJ','INTJ','ADV','ADJ','PART','SYM','DET']

Loading df...


In [18]:
class NutidsrTokenizer():
    def __init__(self) -> None:
        print("Initializing tokenizer...")
        self.tokenize_table = {x: i for i, x in enumerate(unique_pos)}
        print("Tokenizer initialized.")

    def __call__(self, pos_list):
        for pos_string in tqdm(pos_list):
            splitted_pos = pos_string.split()
            numbers = [self.tokenize_table[x] for x in splitted_pos]
            yield [y for x in numbers for y in self._one_hot_encode(x)]
    
    def _one_hot_encode(self, number):
        return [1 if i == number else 0 for i in range(18)]

class NutidsrModel(nn.Module):
    def __init__(self) -> None:
        super(NutidsrModel, self).__init__()
        number_of_unique_pos = 18
        number_of_pos_including_padding = 21
        self.l1 = nn.Linear(number_of_pos_including_padding*number_of_unique_pos, 512)
        self.l2 = nn.Linear(512, 512)
        self.l3 = nn.Linear(512, 256)
        self.l4 = nn.Linear(256, 128)
        self.l5 = nn.Linear(128, 32)
        self.l6 = nn.Linear(32, 1)
        self.activation = nn.LeakyReLU(0.2)
        self.sigmoid = nn.Sigmoid()
        print(self.l1.weight.dtype)

    def forward(self, x):
        x = self.l1(x)
        x = self.activation(x)
        x = self.l2(x)
        x = self.activation(x)
        x = self.l3(x)
        x = self.activation(x)
        x = self.l4(x)
        x = self.sigmoid(x)
        x = self.l5(x)
        x = self.sigmoid(x)
        x = self.l6(x)
        x = self.sigmoid(x)
        return x

In [19]:
tokenizer = NutidsrTokenizer()
print("Tokenizing train...")
x_train_tokenized = list(tokenizer(X_train))
print("Tokenizing test...")
x_test_tokenized = list(tokenizer(X_test))
print("Done Tokenizing.")

Initializing tokenizer...
Tokenizer initialized.
Tokenizing train...


100%|██████████| 1716972/1716972 [01:27<00:00, 19601.62it/s]


Tokenizing test...


100%|██████████| 190775/190775 [00:06<00:00, 29603.96it/s]


Done Tokenizing.


In [32]:
EPOCHS, BATCH_SIZE = 3, 32

model = NutidsrModel()
optimizer = torch.optim.Adam(model.parameters(),lr=0.0002, betas=(0.5, 0.999))
device = "cpu"
loss_fn = nn.MSELoss()
torch.device(device)
model.to(device)
n_steps = len(x_train_tokenized) // BATCH_SIZE
eval_steps = len(x_test_tokenized) // BATCH_SIZE

print("Training model...")
print("INFO: Epochs: ", EPOCHS, ". Batch size: ", BATCH_SIZE, ". Steps pr. epoch: ", n_steps, ". Eval steps: ", eval_steps)
print("Total training steps: ", n_steps*EPOCHS)
print("Number of parameters: ", sum([len(param) for param in model.parameters()]))

torch.float32
Training model...
INFO: Epochs:  3 . Batch size:  32 . Steps pr. epoch:  53655 . Eval steps:  5961
Total training steps:  160965
Number of parameters:  2882


In [33]:
def get_batch(x, y):
    indexes = random.sample(range(len(x)), BATCH_SIZE)
    x_batch = [x[i] for i in indexes]
    y_batch = [y[i] for i in indexes]
    return x_batch, y_batch

def train_model(xb, yb):
    yb = torch.tensor(yb, dtype=torch.float32)
    optimizer.zero_grad()
    xb = torch.tensor(xb, dtype=torch.float32)
    output = model.forward(xb)
    loss = loss_fn(output, yb)
    loss.backward()
    optimizer.step()
    return loss

def test_model(eval_accuracies):
    accuracies = []
    for _ in tqdm(range(eval_steps)):
        xb, yb = get_batch(x_test_tokenized, y_test)
        xb = torch.tensor(xb, dtype=torch.float32)
        output = list(model.forward(xb))
        accuracy = sum([1 if to_binary(o) == y else 0 for (o,y) in zip(output, yb)])/len(output)
        accuracies.append(accuracy)
        del xb, yb, output, accuracy
    eval_accuracies.append(round(sum(accuracies)/len(accuracies)*100, 2))
    print("Eval accuracy: ", round(sum(accuracies)/len(accuracies)*100, 2), "%")
    return eval_accuracies

def to_binary(o):
    return 1 if float(o) > 0.5 else 0

def save_model(epoch):
    os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/FineTuneModels/SimpleNutidsrNN/")
    os.makedirs("simpleNNmodelsPT", exist_ok=True)
    torch.save(model.state_dict(), f"simpleNNmodelsPT/model_{epoch}.pt")

In [39]:
eval_accuracy = []
losses = []

print("Accuracy before training: ")
eval_accuracy = test_model(eval_accuracy)
save_model(0)

Accuracy before training: 


100%|██████████| 5961/5961 [00:08<00:00, 730.48it/s]

Eval accuracy:  57.34 %


In [41]:
for epoch in range(EPOCHS):
    temp_losses = []
    model.train()
    for i in tqdm(range(n_steps)):
        xb, yb = get_batch(x_train_tokenized, y_train)
        xb = torch.tensor(xb)
        loss = train_model(xb, yb)
        temp_losses.append(float(loss))
        if i % 3000 == 0:
            if len(temp_losses) > 0:
                losses.append(sum(temp_losses)/len(temp_losses))
                print(f"Loss at step {i}: {sum(temp_losses)/len(temp_losses)}")
            temp_losses = []
    
    print(f"Done with Epoch {epoch}")
    print("Evaluating...")
    model.eval()

    eval_accuracy = test_model(eval_accuracy)
    save_model(epoch+1)

  0%|          | 0/53655 [00:00<?, ?it/s]/var/folders/pl/7f1cr2657p3bnpdt3bnm_9w00000gn/T/ipykernel_64887/693126525.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xb = torch.tensor(xb, dtype=torch.float32)
/opt/homebrew/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  0%|          | 37/53655 [00:00<04:50, 184.36it/s]

Loss at step 0: 0.2540414035320282


  6%|▌         | 3025/53655 [00:16<04:11, 201.43it/s]

Loss at step 3000: 0.245047217592597


 11%|█         | 6032/53655 [00:30<03:52, 204.73it/s]

...

Loss at step 9000: 0.24467285052438578


 22%|██▏       | 12022/53655 [00:58<05:02, 137.52it/s]

Loss at step 12000: 0.2449960648616155


 23%|██▎       | 12247/53655 [01:00<03:23, 203.20it/s]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(losses)
plt.title("Losses")
plt.show()

In [ ]:
plt.plot(eval_accuracy)
plt.title("Eval accuracy over time")
plt.show()